Dowloading and Isntalling dependencies of spark in the next three coding blocks

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Doing imports required

In [4]:
!pip install -U imbalanced-learn
!pip install seaborn
from pyspark.sql.functions import col, expr, when
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt 
import matplotlib as mpl
import matplotlib.dates as mdates

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 199 kB 5.3 MB/s 
     |████████████████████████████████| 199 kB 22.9 MB/s 
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.8.1
    Uninstalling imbalanced-learn-0.8.1:
      Successfully uninstalled imbalanced-learn-0.8.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Definition of the Attributes according to action required for cleansing

In [5]:
imp_Attributes = ['WifiRssi','dnsLookup','TTFB','AvgJitter','AvgPing','AvgDownloadSpeed','AvgUploadSpeed','LteRssi','WcdmaRssi']
Attributes = ['Latitude','Longitude','@timestamp','Manufacturer','Model','AppName','SdkVersion','AppVersion',
              'AndroidApi','OSVersion','ActiveConnection','WifiRssi','lvl1_name','IP','ASN','ISP','URL','dnsLookup','initialConnection','sslNegotiation',
              'TTFB','fullyLoaded','AvgJitter','AvgPing', 'AvgDownloadSpeed', 'AvgUploadSpeed', 'LteRssi', 'WcdmaRssi']
Record_Removing_Attributes = ['Latitude','Longitude','@timestamp','AppName','SdkVersion','AppVersion','AndroidApi','OSVersion','ActiveConnection','lvl1_name','IP','ASN','ISP','URL','Manufacturer','Model']
Assign_Zero_Attributes = ['WifiRssi','LteRssi','WcdmaRssi','sslNegotiation']
Mean_Assign_Attributes = ['dnsLookup','initialConnection','TTFB','fullyLoaded','AvgJitter','AvgPing','AvgDownloadSpeed','AvgUploadSpeed']
Outliers_Removal =  ['WifiRssi', 'dnsLookup', 'initialConnection', 'sslNegotiation', 'TTFB', 
                     'fullyLoaded', 'AvgJitter', 'AvgPing', 'AvgDownloadSpeed', 'AvgUploadSpeed', 'LteRssi', 'WcdmaRssi']

Reading the contents of the json file and separating for all media Platforms. Please paste your link/address here.

In [6]:
# Please paste the file link here.
DF21 = spark.read.json('/content/1splitted_2021new.json')
DF21 = DF21.select(Attributes)
DF21 = DF21.withColumn("WifiRssi",DF21.WifiRssi.cast('double'))
DF_facebook = DF21.where(DF21.URL =="https://www.facebook.com")
DF_twitter = DF21.where(DF21.URL =="https://mobile.twitter.com/explore")
DF_instagram = DF21.where(DF21.URL =="https://www.instagram.com")
del DF21

DF22 = spark.read.json('/content/1splitted_2022new.json')
DF22 = DF22.select(Attributes)
DF22 = DF22.withColumn("WifiRssi",DF22.WifiRssi.cast('double'))
DF22_facebook = DF22.where(DF22.URL =="https://www.facebook.com")
DF22_twitter = DF22.where(DF22.URL =="https://mobile.twitter.com/explore")
DF22_instagram = DF22.where(DF22.URL =="https://www.instagram.com")
del DF22

DF_facebook = DF_facebook.unionByName(DF22_facebook, allowMissingColumns=True)
DF_twitter = DF_twitter.unionByName(DF22_twitter, allowMissingColumns=True)
DF_instagram = DF_instagram.unionByName(DF22_instagram, allowMissingColumns=True)
# print(DF_instagram.count())

del DF22_facebook,DF22_instagram,DF22_twitter

Function for Removing Duplicates from the dataset samples.

In [7]:
def Remove_Duplicates (df):
  print("Checking and Removing Duplicates\n","Samples before removing:",df.count())
  df = df.distinct()
  print("Samples after removing Duplicates:",df.count())
  return df
DF_facebook = Remove_Duplicates (DF_facebook)
DF_twitter = Remove_Duplicates (DF_twitter)
DF_instagram = Remove_Duplicates (DF_instagram)

Checking and Removing Duplicates
 Samples before removing: 2169
Samples after removing Duplicates: 2111
Checking and Removing Duplicates
 Samples before removing: 2194
Samples after removing Duplicates: 2134
Checking and Removing Duplicates
 Samples before removing: 2133
Samples after removing Duplicates: 2080


Function for Record Removing Attributes

In [8]:
def Remove_Records (df,Record_Removing_Attributes):
  df = df.na.drop(subset=Record_Removing_Attributes)
  return df
DF_facebook = Remove_Records (DF_facebook,Record_Removing_Attributes)
DF_twitter = Remove_Records (DF_twitter,Record_Removing_Attributes)
DF_instagram = Remove_Records (DF_instagram,Record_Removing_Attributes)

In [9]:
Facebook = DF_facebook.toPandas()
Twitter = DF_twitter.toPandas()
Instagram = DF_instagram.toPandas()
Facebook.to_csv('/content/Facebook.csv')
Twitter.to_csv('/content/Twitter.csv')
Instagram.to_csv('/content/Instagram.csv')

In [ ]:
DF_facebook.select(Outliers_Removal).dtypes

[('WifiRssi', 'double'),
 ('dnsLookup', 'bigint'),
 ('initialConnection', 'bigint'),
 ('sslNegotiation', 'bigint'),
 ('TTFB', 'bigint'),
 ('fullyLoaded', 'bigint'),
 ('AvgJitter', 'double'),
 ('AvgPing', 'double'),
 ('AvgDownloadSpeed', 'double'),
 ('AvgUploadSpeed', 'double'),
 ('LteRssi', 'bigint'),
 ('WcdmaRssi', 'bigint')]

In [ ]:
abc = 'WifiRssi'
abc
# DF_facebook.abc

'WifiRssi'

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F, Window

In [ ]:
df = DF_facebook
a = df.where((df.WifiRssi >= -108.0 ) & (df.WifiRssi <= -4.0 )).count()
print(a)

1367


In [ ]:
# Facebook = find_outliers(DF_facebook)
df = DF_facebook.select(Outliers_Removal)
print(df.count())
param = 1
for column in Outliers_Removal:
        # cl = df[column]
        # pdf1 = df.toPandas()
        # print(pdf1.describe(include='all'))

        # print(df.count())
        less_Q1 = 'less_Q1_{}'.format(column)
        more_Q3 = 'more_Q3_{}'.format(column)
        Q1 = 'Q1_{}'.format(column)
        Q3 = 'Q3_{}'.format(column)

        # Q1 : First Quartile ., Q3 : Third Quartile
        Q1 = df.approxQuantile(column,[0.25],relativeError=0)
        Q3 = df.approxQuantile(column,[0.75],relativeError=0)
        if (len(Q1)== 0 or len(Q3)==0):
          # print(df)
          # pdf1 = df.toPandas()
          # print(pdf1.describe(include='all'))
          continue
        # IQR : Inter Quantile Range
        # We need to define the index [0], as Q1 & Q3 are a set of lists., to perform a mathematical operation
        # Q1 & Q3 are defined seperately so as to have a clear indication on First Quantile & 3rd Quantile
        IQR = Q3[0] - Q1[0]
        
        #selecting the data, with -1.5*IQR to + 1.5*IQR., where param = 1.5 default value
        less_Q1 =  Q1[0] - param*IQR
        more_Q3 =  Q3[0] + param*IQR

        
        cl = df.where((col(column) >= less_Q1) & (col(column) <= more_Q3))
        mean = cl.agg({column: 'mean'}).collect()[0][0]

        print(column,"Lower Limit: ",less_Q1,"  Upper Limit: ",more_Q3,"   Mean: ",mean)
        # df = df.withColumn(column, F.when((df[column] > more_Q3) | (df[column] < less_Q1), mean).otherwise(F.col(column)))
        # df = df.withColumn(column, F.when((df[column] > more_Q3) | (df[column] < less_Q1), mean).otherwise(F.col(column)))
        df = df.where((col(column) >= less_Q1) & (col(column) <= more_Q3))
        
        

2750
WifiRssi Lower Limit:  -95.0   Upper Limit:  -17.0    Mean:  -55.82763744427935
dnsLookup Lower Limit:  -42.0   Upper Limit:  84.0    Mean:  17.081720430107527
initialConnection Lower Limit:  -223.0   Upper Limit:  446.0    Mean:  105.13907284768212
sslNegotiation Lower Limit:  57.0   Upper Limit:  228.0    Mean:  140.67088607594937
TTFB Lower Limit:  271.0   Upper Limit:  1030.0    Mean:  609.9109195402299
fullyLoaded Lower Limit:  417.0   Upper Limit:  4197.0    Mean:  2171.3665594855306
AvgJitter Lower Limit:  -10.700000000000003   Upper Limit:  57.400000000000006    Mean:  22.763250883392228
AvgPing Lower Limit:  -5.600000000000001   Upper Limit:  53.2    Mean:  23.617407407407402
AvgDownloadSpeed Lower Limit:  -0.4350000000000023   Upper Limit:  53.42100000000001    Mean:  24.1521851851852
AvgUploadSpeed Lower Limit:  -5.968999999999998   Upper Limit:  27.483999999999995    Mean:  10.336696428571434


In [ ]:
Facebook = DF_facebook.toPandas()
del DF_facebook
Twitter = DF_twitter.toPandas()
del DF_twitter
Isntagram = DF_instagram.toPandas()
del DF_instagram

In [ ]:
Facebook.columns(Outliers_Removal)

TypeError: ignored

In [ ]:
def remove_outliers(df,columns,n_std):
    for col in columns:
        print('Working on column: {}'.format(col))
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

In [ ]:
mean = Twitter["WifiRssi"].mean()

TypeError: ignored

In [ ]:
print(len(Facebook))
# Facebook = remove_outliers(Facebook,'dnsLookup',3)
Facebook = remove_outliers(Facebook,Outliers_Removal,3)
print(len(Facebook))

2750
Working on column: WifiRssi


TypeError: ignored

In [ ]:
def find_outliers(df):

    # Identifying the numerical columns in a spark dataframe
    numeric_columns = [column[0] for column in df.dtypes if column[1]=='int']

    # Using the `for` loop to create new columns by identifying the outliers for each feature
    for column in numeric_columns:

        less_Q1 = 'less_Q1_{}'.format(column)
        more_Q3 = 'more_Q3_{}'.format(column)
        Q1 = 'Q1_{}'.format(column)
        Q3 = 'Q3_{}'.format(column)

        # Q1 : First Quartile ., Q3 : Third Quartile
        Q1 = df.approxQuantile(column,[0.25],relativeError=0)
        Q3 = df.approxQuantile(column,[0.75],relativeError=0)
        
        # IQR : Inter Quantile Range
        # We need to define the index [0], as Q1 & Q3 are a set of lists., to perform a mathematical operation
        # Q1 & Q3 are defined seperately so as to have a clear indication on First Quantile & 3rd Quantile
        IQR = Q3[0] - Q1[0]
        
        #selecting the data, with -1.5*IQR to + 1.5*IQR., where param = 1.5 default value
        less_Q1 =  Q1[0] - 1.5*IQR
        more_Q3 =  Q3[0] + 1.5*IQR
        
        isOutlierCol = 'is_outlier_{}'.format(column)
        
        df = df.withColumn(isOutlierCol,f.when((df[column] > more_Q3) | (df[column] < less_Q1), 1).otherwise(0))
    

    # Selecting the specific columns which we have added above, to check if there are any outliers
    selected_columns = [column for column in df.columns if column.startswith("is_outlier")]

    # Adding all the outlier columns into a new colum "total_outliers", to see the total number of outliers
    df = df.withColumn('total_outliers',sum(df[column] for column in selected_columns))

    # Dropping the extra columns created above, just to create nice dataframe., without extra columns
    # df = df.drop(*[column for column in df.columns if column.startswith("is_outlier")])

    return df
    

In [ ]:
print(DF_instagram)

+----------+----------+--------------------+------------+---------+--------------------+----------+----------+----------+---------+----------------+--------+--------------------+---------------+-----+-------------------+--------------------+---------+-----------------+--------------+----+-----------+---------+-------+----------------+--------------+-------+---------+
|  Latitude| Longitude|          @timestamp|Manufacturer|    Model|             AppName|SdkVersion|AppVersion|AndroidApi|OSVersion|ActiveConnection|WifiRssi|           lvl1_name|             IP|  ASN|                ISP|                 URL|dnsLookup|initialConnection|sslNegotiation|TTFB|fullyLoaded|AvgJitter|AvgPing|AvgDownloadSpeed|AvgUploadSpeed|LteRssi|WcdmaRssi|
+----------+----------+--------------------+------------+---------+--------------------+----------+----------+----------+---------+----------------+--------+--------------------+---------------+-----+-------------------+--------------------+---------+---------

In [ ]:
pdf1 = DF.toPandas()
pdf1.describe(include='all')

,Latitude,Longitude,@timestamp,Manufacturer,Model,AppName,SdkVersion,AppVersion,AndroidApi,OSVersion,...,initialConnection,sslNegotiation,TTFB,fullyLoaded,AvgJitter,AvgPing,AvgDownloadSpeed,AvgUploadSpeed,LteRssi,WcdmaRssi
count,19964.000000,19964.000000,20002,20002,20002,20002,20002,20002,20002,20002,...,8272.000000,5091.000000,8272.000000,8287.000000,18889.000000,19128.000000,19152.000000,19120.000000,2290.000000,198.000000
unique,NaN,NaN,19330,36,430,4,19,26,14,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,2021-01-01T12:30:26+00:00,samsung,SM-N960F,com.speedchecker.bh.weather,4.2.104,1.1.3,29,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,5,12976,1050,14187,6229,4749,5626,5626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,26.184342,50.564534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,208.984163,230.948929,982.820479,4105.824303,22.593615,27.720065,25.947132,12.500999,-73.461135,-77.914141
std,0.065657,0.052469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,440.193670,265.444883,1201.008228,3791.956908,22.294251,24.191485,23.339063,9.527785,16.608396,15.332117
min,25.617760,50.438125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,24.000000,69.000000,0.000000,3.000000,0.050000,0.191000,-113.000000,-113.000000
25%,26.135210,50.522740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,140.000000,584.000000,2037.500000,10.400000,15.800000,13.530000,6.244000,-87.000000,-91.000000
50%,26.199980,50.570635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,198.000000,179.000000,739.000000,2925.000000,18.500000,22.600000,20.445000,10.131000,-71.000000,-77.000000
75%,26.231175,50.600870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,288.000000,250.000000,1005.000000,4550.500000,28.000000,31.400000,32.492000,15.584250,-59.000000,-65.000000


In [ ]:
import numpy as np
from scipy import stats
pdf1 = pdf1[(np.abs(stats.zscore(pdf1)) < 3).all(axis=1)]  # Will remove outliers 3 Standard Deviations from mean
# pdf1 = DF.toPandas()
pdf1.describe(include='all')

KeyboardInterrupt: ignored

In [ ]:
# Mean Value Assignment
for x in Mean_Assign_Attributes:
  df = df.fillna(value = DF.agg({x: 'mean'}).collect()[0][0],subset = x)

-223.95973099983794


Identifying the Important Attributes for the class labeling.

In [ ]:
# imp_Attributes = ['DownloadSpeed','UploadSpeed','Ping','TTFB']
imp_Attributes = ['WifiRssi','dnsLookup','TTFB','AvgJitter','AvgPing','AvgDownloadSpeed','AvgUploadSpeed','LteRssi','WcdmaRssi']
Attributes = ['Latitude','Longitude','@timestamp','Manufacturer','Model','AppName','SdkVersion','AppVersion',
              'AndroidApi','OSVersion','ActiveConnection','WifiRssi','lvl1_name','IP','ASN','ISP','URL','dnsLookup','initialConnection','sslNegotiation',
              'TTFB','fullyLoaded','AvgJitter','AvgPing', 'AvgDownloadSpeed', 'AvgUploadSpeed', 'LteRssi', 'WcdmaRssi']

Defining Function to calculate Median. Finding medians of required attributes.

In [ ]:
def calculate_Median(tdf,At):
  tdf = tdf.select(At)
  tdf = tdf.na.drop(subset=[At])
  count = tdf.count()
  tdf = tdf.sort(At)
  if count % 2 ==0:
    Median = (tdf.select(At).collect()[int(count/2)][0] + tdf.select(At).collect()[int(count/2)-1][0])/2
  else:
    Median = tdf.select(At).collect()[int((count-1)/2)][0]
  return Median

Med_DNS = calculate_Median(DF,imp_Attributes[0])
Med_UPS = calculate_Median(DF,imp_Attributes[1])
Med_Ping = calculate_Median(DF,imp_Attributes[2])
Med_TTFB = calculate_Median(DF,imp_Attributes[3])

In [ ]:
print (' Median DNS: ',Med_DNS,"\n",'Median UPS: ',Med_UPS,"\n",'Median Ping: ',Med_Ping,"\n",'Median TTFB: ',Med_TTFB )

All the Attributes to be used are defined here in this list.

In [ ]:
# Attributes = ['Latitude','Longitude','@timestamp','Manufacturer','Model','AppName','SdkVersion','AppVersion',
#               'AndroidApi','OSVersion','ActiveConnection','WifiBSSID','WifiRssi','WifiLinkSpeed',
#               'lvl1_name','IP','ASN','ISP','URL','dnsLookup','initialConnection','sslNegotiation',
#               'TTFB','fullyLoaded','PageloadSpeed','loadEventEnd','documentComplete','Ping',
#               'DownloadSpeed','UploadSpeed','LteRssi','WcdmaRssi','GsmRssi']
Attributes = ['Latitude','Longitude','@timestamp','Manufacturer','Model','AppName','SdkVersion','AppVersion',
              'AndroidApi','OSVersion','ActiveConnection','WifiRssi','lvl1_name','IP','ASN','ISP','URL','dnsLookup','initialConnection','sslNegotiation',
              'TTFB','fullyLoaded','AvgJitter','AvgPing', 'AvgDownloadSpeed', 'AvgUploadSpeed', 'LteRssi', 'WcdmaRssi']

In [ ]:
df = DF.select(Attributes)

Visualizing the Attributes in our data to make sure we are using required ones.

In [ ]:
df.columns

Defining the Attributes according to the Action required for NULL values.

In [ ]:
Record_Removing_Attributes = ['Latitude','Longitude','@timestamp','AppName','SdkVersion','AppVersion','AndroidApi','OSVersion','ActiveConnection','lvl1_name','IP','ASN','ISP','URL','Manufacturer','Model']
Assign_Zero_Attributes = ['WifiRssi','LteRssi','WcdmaRssi','sslNegotiation']

Droping rows of NULL values against record removing attributes and with google URL

In [ ]:
df = df.na.drop(subset=Record_Removing_Attributes)
df = df[df.ActiveConnection != ""] # There were some values in this coulum that were empty but not assigned NULL position.
df=df.where(df.URL !="https://www.google.com.bh")
print("Truncated df count will be :", df.count())

In [ ]:
print('Before replacing values')
df

Replacement of NULL values with 0 and Medians accordingly

In [ ]:
df = df.fillna(value = Med_DNS ,subset = imp_Attributes[0])
df = df.fillna(value = Med_UPS ,subset = imp_Attributes[1])
df = df.fillna(value = Med_Ping ,subset = imp_Attributes[2])
df = df.fillna(value = Med_TTFB ,subset = imp_Attributes[3])
df = df.fillna(value=0,subset=Assign_Zero_Attributes)
df = df.fillna(value='0',subset=Assign_Zero_Attributes_str)
print('After replacing values')
df


Counting Acceptable values in data

In [ ]:
df.where((df.DownloadSpeed >= 1.5) & (df.UploadSpeed >= 0.5) & (df.Ping <=20) & (df.TTFB <=1200)).count()

Adding QoE Coulum

In [ ]:
df3 = df.withColumn("QoE", when(((df.DownloadSpeed >= 1.5) & (df.UploadSpeed >= 0.5) & (df.Ping <=20) & (df.TTFB <=1200)),1)
                                 .otherwise(0))

Understanding the storage type of data inside the coulums

In [ ]:
df3.printSchema()

Extracting name of coulums containing string data

In [ ]:
String_Cols = [item[0] for item in df3.dtypes if item[1].startswith('string')]

In [ ]:
print(String_Cols)

'gdf' contains frequency of each type of entry in string data frames.

In [ ]:
gdf = pd.DataFrame()
for x in String_Cols:
  t = df3.groupBy(x).count().toPandas()
  gdf = pd.concat([gdf,t],axis = 1)

Saving Frequency information of string coulums in "String_Frequency.csv" file.

In [ ]:
gdf.to_csv('/content/String_Frequency.csv')

Saving actual data after Cleansing to "After_Cleansing.csv" file

In [ ]:
pdf = df3.toPandas()

In [ ]:
pdf.to_csv('/content/After_Cleansing.csv')

It is saving the description of Data Distribution of all the coulums to "Distribution.csv" file 

In [ ]:
pdf.describe(include='all').to_csv('/content/Distribution.csv')

Code for using ADASYN technique of Oversampling for Data Balancing

In [ ]:
def makeOverSamplesADASYN(X,y):
 #input DataFrame
 #X →Independent Variable in DataFrame\
 #y →dependent Variable in Pandas DataFrame format
 from imblearn.over_sampling import ADASYN 
 sm = ADASYN()
 X, y = sm.fit_resample(X, y)
 return(X,y)

In [ ]:
P_df = df.select(imp_Attributes).toPandas()
Y_df = df3.select('QoE').toPandas()
P_df,Y_df = makeOverSamplesADASYN(P_df,Y_df)

In [ ]:
result = pd.concat([P_df,Y_df])

In [ ]:
result.to_csv('/content/After_ADASYN.csv')

Sample code for multi conditional work on spark

In [ ]:
# df2 = df.withColumn("QoE", when(((df.DownloadSpeed >= 1.5) & (df.DownloadSpeed < 5) & (df.UploadSpeed >= 0.5) & (df.UploadSpeed < 1.5) & (df.Ping >50)& (df.Ping <=100) & (df.TTFB >800)& (df.TTFB <=1200)) ,"A")
#                                  .when((df.DownloadSpeed >= 5) & (df.UploadSpeed >= 1.5) & (df.Ping <=50) & (df.TTFB <=800),"E")
#                                  .otherwise("NA"))

Code for removing Outliers

In [ ]:
import numpy as np
from scipy import stats
df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # Will remove outliers 3 Standard Deviations from mean

Code for Normalization of values in the data frame

In [ ]:
# copy the data
df_min_max_scaled = df.copy()
  
# apply normalization techniques
for column in df_min_max_scaled.columns:
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())  